<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Запуск-Spark" data-toc-modified-id="Запуск-Spark-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Запуск Spark</a></span></li><li><span><a href="#Обзор" data-toc-modified-id="Обзор-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обзор</a></span></li><li><span><a href="#RDD" data-toc-modified-id="RDD-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>RDD</a></span></li><li><span><a href="#hist" data-toc-modified-id="hist-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>hist</a></span></li><li><span><a href="#Завершение" data-toc-modified-id="Завершение-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Завершение</a></span></li></ul></div>

### Запуск Spark

In [61]:
import json

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [2]:
from pyspark import SparkContext, SparkConf

conf = SparkConf()
conf.set("spark.app.name", "alexander.prutko Spark RDD app") 

sc = SparkContext(conf=conf)

In [3]:
sc

<SparkContext master=yarn appName=alexander.prutko Spark RDD app>

### Обзор

In [4]:
!hdfs dfs -ls /labs/lab06data/ml-100k

Found 23 items
-rw-r--r--   2 hdfs hdfs       6403 2020-09-30 12:16 /labs/lab06data/ml-100k/README
-rw-r--r--   2 hdfs hdfs        716 2020-09-30 12:16 /labs/lab06data/ml-100k/allbut.pl
-rw-r--r--   2 hdfs hdfs        643 2020-09-30 12:16 /labs/lab06data/ml-100k/mku.sh
-rw-r--r--   2 hdfs hdfs    1979173 2020-09-30 12:16 /labs/lab06data/ml-100k/u.data
-rw-r--r--   2 hdfs hdfs        202 2020-09-30 12:16 /labs/lab06data/ml-100k/u.genre
-rw-r--r--   2 hdfs hdfs         36 2020-09-30 12:16 /labs/lab06data/ml-100k/u.info
-rw-r--r--   2 hdfs hdfs     236344 2020-09-30 12:16 /labs/lab06data/ml-100k/u.item
-rw-r--r--   2 hdfs hdfs        193 2020-09-30 12:16 /labs/lab06data/ml-100k/u.occupation
-rw-r--r--   2 hdfs hdfs      22628 2020-09-30 12:16 /labs/lab06data/ml-100k/u.user
-rw-r--r--   2 hdfs hdfs    1586544 2020-09-30 12:16 /labs/lab06data/ml-100k/u1.base
-rw-r--r--   2 hdfs hdfs     392629 2020-09-30 12:16 /labs/lab06data/ml-100k/u1.test
-rw-r--r--   2 hdfs hdfs    1583948 2020-09-30 12

In [ ]:
!hdfs dfs -head /labs/lab06data/ml-100k/u.data

In [18]:
!hdfs dfs -cat /labs/lab06data/ml-100k/u.item | grep -P "^96\|"

96|Terminator 2: Judgment Day (1991)|01-Jan-1991||http://us.imdb.com/M/title-exact?Terminator%202:%20Judgment%20Day%20(1991)|0|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|1|0|0


Мне попался "Терминатор 2: Судный День"

### RDD

In [25]:
data = sc.textFile("/labs/lab06data/ml-100k/u.data")

In [27]:
data_splitted = data.map(lambda x: x.split("\t"))

In [28]:
data_splitted.take(5)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596']]

### hist

In [53]:
hist_film = (data_splitted.filter(lambda x: x[1] == "96")
    .map(lambda x: (x[2], 1))
    .countByKey()
    .items())

In [57]:
hist_all = (data_splitted
    .map(lambda x: (x[2], 1))
    .countByKey()
    .items())

In [64]:
result = {"hist_film": [i[1] for i in sorted(hist_film, key=lambda x: x[0])],
          "hist_all" : [i[1] for i in sorted(hist_all,  key=lambda x: x[0])]}
result

{'hist_film': [6, 20, 43, 123, 103],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [66]:
with open("lab06.json", "w") as f:
    f.write(json.dumps(result))

### Завершение

In [67]:
sc.stop()

## Code review

In [ ]:
Всё чётко и лаконично.
Единственное можно было data_splitted не выделять в отдельную переменную, а сразу добавить map в hist_film/hist_all:
data_splitted = data.map(lambda x: x.split("\t"))
+
hist_film = (data_splitted.filter(lambda x: x[1] == "96")
    .map(lambda x: (x[2], 1))
    .countByKey()
    .items())

Превращаем в:
hist_film = (data.map(lambda x: x.split("\t"))
                 .filter(lambda x: x[1] == "96")
                 .map(lambda x: (x[2], 1))
                 .countByKey()
                 .items())

Но это чисто косметика.